# Solving the problem of 3-coloring using ASP

This is a small example to show how you can encode the problem of [3-coloring](https://en.wikipedia.org/wiki/Graph_coloring#Vertex_coloring) into ASP and using an ASP solver to solve the problem.

We will use [clingo](https://potassco.org/clingo/) in Python for this. Have a look at [these instructions](asp.ipynb) to see how you can use clingo from Python.

Let's start off with an empty string, that we will fill with an answer set program.

In [1]:
import clingo

asp_code = "";

## 3-coloring

In the problem of 3-coloring, you are given as input a finite (undirected) [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) $G = (V,E)$, consisting of a set $V$ of *vertices* (or *nodes*) and a set $E$ of *edges*. Each edge $e \in E$ consists of a set $\{v_1,v_2\}$ of exactly two vertices in $V$.

A *coloring* of a graph $G = (V,E)$ is a function $\mu : V \rightarrow C$ that assigns to each vertex $v \in V$ a color $\mu(v) \in C$. The coloring $\mu$ is called *proper* if for every edge $\{v_1,v_2\} \in E$, the coloring assigns different colors to the two endpoints of the edge—that is, $\mu(v_1) \neq \mu(v_2)$.

In the problem of 3-coloring, the question is to decide if there exists a proper coloring $\mu : V \rightarrow \{1,2,3\}$ of the graph that only uses three colors.

## Encoding graphs in ASP

To encode this problem in ASP, we first show how to encode a given graph in ASP.

Take the following example graph $G = (V,E)$, where $V = \{v_1,v_2,v_3,v_4\}$ and $E = \{ \{v_1,v_2\}, \{v_1,v_3\}, \{v_2,v_3\}, \{v_2,v_4\}, \{v_3,v_4\} \}$. We can encode this in ASP using predicates `vertex/1` and `edge/2` as follows.

In [2]:
asp_code += """
    vertex(v1).
    vertex(v2).
    vertex(v3).
    vertex(v4).
    edge(v1,v2).
    edge(v1,v3).
    edge(v2,v3).
    edge(v2,v4).
    edge(v3,v4).
""";

## Guessing a 3-coloring

We will use a guess-and-check approach to model the problem of 3-coloring. In particular, we will add some rules to the answer set program that will guess a coloring, and then after that we will add some constraints that ensure that the coloring that we guessed is proper.

We guess the 3-coloring as follows:

In [3]:
asp_code += """
    color(V,1) :- vertex(V), not color(V,2), not color(V,3).
    color(V,2) :- vertex(V), not color(V,1), not color(V,3).
    color(V,3) :- vertex(V), not color(V,1), not color(V,2).
"""

## Verifying that the coloring is proper

We now add constraints expressing that the guessed color for any two vertices that are connected by an edge must be different:

In [4]:
asp_code += """
    :- edge(V1,V2), color(V1,C), color(V2,C).
"""

## Showing the coloring

Finally, we add a `#show` statement that indicates that we are only interested in the predicate `color/2`, and we ask clingo to print out all proper 3-colorings of the graph.

In [5]:
asp_code += """
    #show color/2.
"""

control = clingo.Control();
control.add("base", [], asp_code);
control.ground([("base", [])])

def on_model(model):
    print(model.symbols(shown=True));

control.configuration.solve.models = 0;
answer = control.solve(on_model=on_model)

if answer.satisfiable == True:
    print("The graph is 3-colorable");
else:
    print("The graph is not 3-colorable");

[color(v1,3), color(v2,1), color(v3,2), color(v4,3)]
[color(v1,2), color(v2,1), color(v3,3), color(v4,2)]
[color(v1,1), color(v2,3), color(v3,2), color(v4,1)]
[color(v1,2), color(v2,3), color(v3,1), color(v4,2)]
[color(v1,1), color(v2,2), color(v3,3), color(v4,1)]
[color(v1,3), color(v2,2), color(v3,1), color(v4,3)]
The graph is 3-colorable
